# Chronos-2: Amazon's Universal Time Series Foundation Model

## 📚 Overview

**Chronos-2** is Amazon's encoder-only transformer foundation model for time series forecasting.

### Key Features
- **Pre-trained on 100,000+ time series** from diverse domains
- **Encoder-only architecture** (BERT-style for time series)
- **Probabilistic forecasting** with uncertainty quantification
- **Multiple quantile predictions** (10th, 50th, 90th percentiles)
- **Zero-shot forecasting** capability

### Chronos-2 vs TimesFM

| Feature | Chronos-2 | TimesFM |
|---------|-----------|----------|
| Architecture | Encoder-only (BERT-style) | Decoder-only (GPT-style) |
| Pre-training | 100K+ time series | 100B time points |
| Approach | Masked modeling | Autoregressive generation |
| Uncertainty | ✅ Quantile forecasts | Limited |

### When to Use Chronos-2
- ✅ Need uncertainty quantification (confidence intervals)
- ✅ Probabilistic forecasting for risk assessment
- ✅ Limited training data (zero-shot capability)
- ✅ Diverse time series types

## 🔧 Setup

In [ ]:
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from models import ChronosTimeSeriesModel

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Check if Chronos is available
try:
    from chronos import ChronosPipeline
    print("✅ Chronos package available")
except ImportError:
    print("⚠️  Chronos not found")
    print("   Install with: pip install git+https://github.com/amazon-science/chronos-forecasting.git")

## 📊 Load and Prepare Data

In [ ]:
# Load your data
data_path = '../data/train.csv'

if os.path.exists(data_path):
    df = pd.read_csv(data_path)
    print(f"Data shape: {df.shape}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())
else:
    print(f"Data file not found at {data_path}")
    print("Creating synthetic data for demonstration...")
    
    # Create synthetic time series with uncertainty
    n_points = 1000
    dates = pd.date_range('2020-01-01', periods=n_points, freq='D')
    
    # Pattern with increasing variance
    trend = np.linspace(100, 200, n_points)
    seasonality = 25 * np.sin(2 * np.pi * np.arange(n_points) / 365)
    # Heteroscedastic noise (variance increases over time)
    noise_std = np.linspace(3, 10, n_points)
    noise = np.random.normal(0, noise_std)
    
    df = pd.DataFrame({
        'date': dates,
        'target': trend + seasonality + noise
    })
    print(f"Created synthetic data with shape: {df.shape}")
    print(df.head())

## 📈 Visualize the Data

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df['target'].values, linewidth=1.5)
plt.title('Time Series Data (Note: Increasing Variance)', fontsize=14, fontweight='bold')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.show()

## 🔄 Prepare Data for Chronos-2

In [ ]:
# Extract target values
target_col = 'target'
data = df[target_col].values

print(f"Data shape: {data.shape}")

# Train/validation split
split_idx = int(len(data) * 0.8)
train_data = data[:split_idx]
val_data = data[split_idx:]

print(f"Train data: {len(train_data)} points")
print(f"Validation data: {len(val_data)} points")

## 🏗️ Initialize Chronos-2 Model

### Model Sizes
- **tiny**: ~8M parameters (fastest, lowest accuracy)
- **mini**: ~20M parameters (fast, good for testing)
- **small**: ~46M parameters (balanced) ⭐ Recommended
- **base**: ~200M parameters (high accuracy)
- **large**: ~710M parameters (best accuracy, slowest)

### Key Parameters
- **prediction_length**: How far ahead to forecast
- **num_samples**: Number of sample paths (for probabilistic forecasting)

In [ ]:
# Initialize Chronos-2
model = ChronosTimeSeriesModel(model_size='small')  # or 'tiny', 'mini', 'base', 'large'

print("Loading Chronos-2 model...")
print("This may take a moment on first run (downloading pre-trained weights)")
model.load_model()

print("\n" + "="*50)
print("✅ Chronos-2 Model Loaded!")
print("="*50)
print(f"Model size: small")
print(f"Pre-trained: Yes (100K+ time series)")
print(f"Zero-shot ready: Yes")

## 🔮 Make Zero-Shot Point Predictions

In [ ]:
# Make predictions
context_length = 96
prediction_length = 24

context = val_data[:context_length]

print(f"Making predictions with:")
print(f"  Context: {len(context)} points")
print(f"  Forecast horizon: {prediction_length} steps")

# Generate forecasts (median prediction)
forecasts = model.predict(
    context=context,
    prediction_length=prediction_length,
    num_samples=20  # More samples = better uncertainty estimates
)

print(f"\nForecast shape: {forecasts.shape}")
print(f"First 10 predictions: {forecasts[:10]}")

## 📊 Probabilistic Forecasting with Uncertainty

In [ ]:
# Get quantile forecasts (uncertainty intervals)
quantiles = [0.1, 0.5, 0.9]  # 10th, 50th (median), 90th percentiles

print("Generating probabilistic forecasts...")
quantile_forecasts = model.predict_quantiles(
    context=context,
    prediction_length=prediction_length,
    quantiles=quantiles
)

print(f"\nQuantile forecasts shape: {len(quantile_forecasts)}")
print(f"Quantiles: {quantiles}")

## 📈 Visualize Probabilistic Forecast with Confidence Intervals

In [ ]:
# Get actual future values
actual_future = val_data[context_length:context_length + prediction_length]

# Extract quantiles
q10 = quantile_forecasts[0]
q50 = quantile_forecasts[1]  # Median
q90 = quantile_forecasts[2]

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Historical context
ax1.plot(range(len(context)), context, label='Historical Context', linewidth=2, color='blue')
ax1.axvline(len(context) - 1, color='red', linestyle='--', alpha=0.5, label='Forecast Start')
ax1.set_title('Input Context for Chronos-2', fontsize=14, fontweight='bold')
ax1.set_xlabel('Time Step', fontsize=12)
ax1.set_ylabel('Value', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Probabilistic forecast
forecast_range = range(len(context), len(context) + prediction_length)

# Historical context (faded)
ax2.plot(range(len(context)), context, label='Historical', linewidth=2, alpha=0.3, color='blue')

# Actual future
ax2.plot(forecast_range, actual_future, 'g-', label='Actual Future', linewidth=2.5, marker='o', markersize=5)

# Median forecast
ax2.plot(forecast_range, q50, 'r--', label='Forecast (Median)', linewidth=2.5, marker='s', markersize=4)

# Confidence intervals (80% prediction interval)
ax2.fill_between(forecast_range, q10, q90, alpha=0.3, color='red', label='80% Confidence Interval')

ax2.axvline(len(context) - 1, color='red', linestyle='--', alpha=0.5)
ax2.set_title('Chronos-2 Probabilistic Forecast with Uncertainty', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time Step', fontsize=12)
ax2.set_ylabel('Value', fontsize=12)
ax2.legend(fontsize=11, loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate coverage (how many actual values fall within 80% interval)
within_interval = np.sum((actual_future >= q10) & (actual_future <= q90))
coverage = (within_interval / len(actual_future)) * 100

print(f"\n📊 Uncertainty Quantification:")
print(f"Prediction Interval: 10th-90th percentile (80% interval)")
print(f"Coverage: {coverage:.1f}% of actual values fall within interval")
print(f"Expected coverage: ~80%")

# Forecast accuracy
mae = np.mean(np.abs(actual_future - q50))
print(f"\nMedian Forecast MAE: {mae:.4f}")

## 🎲 Sample Multiple Forecast Paths

In [ ]:
# Generate multiple sample paths
print("Generating 50 sample forecast paths...")

num_paths = 50
sample_paths = []

for i in range(num_paths):
    forecast = model.predict(context, prediction_length, num_samples=1)
    sample_paths.append(forecast)

sample_paths = np.array(sample_paths)

# Plot all paths
plt.figure(figsize=(15, 6))

# Historical
plt.plot(range(len(context)), context, 'b-', label='Historical', linewidth=2)

# All sample paths (transparent)
forecast_range = range(len(context), len(context) + prediction_length)
for i, path in enumerate(sample_paths):
    plt.plot(forecast_range, path, 'r-', alpha=0.1, linewidth=0.5)

# Actual future
plt.plot(forecast_range, actual_future, 'g-', label='Actual', linewidth=3, marker='o', markersize=6)

# Mean of all paths
mean_path = sample_paths.mean(axis=0)
plt.plot(forecast_range, mean_path, 'r--', label='Mean Forecast', linewidth=2.5, marker='s', markersize=5)

plt.axvline(len(context) - 1, color='black', linestyle='--', alpha=0.3)
plt.title(f'Chronos-2: {num_paths} Sample Forecast Paths', fontsize=14, fontweight='bold')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nForecast uncertainty:")
print(f"  Mean: {mean_path.mean():.2f}")
print(f"  Std Dev: {sample_paths.std(axis=0).mean():.2f}")
print(f"  Min prediction: {sample_paths.min():.2f}")
print(f"  Max prediction: {sample_paths.max():.2f}")

## 🔬 Compare Different Context Lengths

In [ ]:
# Test different context lengths
context_lengths = [32, 64, 96, 128]
results = {}

print("Testing different context lengths...")
for ctx_len in context_lengths:
    if ctx_len > len(val_data) - prediction_length:
        continue
    
    context = val_data[:ctx_len]
    forecast = model.predict(context, prediction_length, num_samples=10)
    
    actual = val_data[ctx_len:ctx_len + prediction_length]
    mae = np.mean(np.abs(actual - forecast))
    results[ctx_len] = mae
    print(f"Context length {ctx_len:3d}: MAE = {mae:.4f}")

# Plot
if results:
    plt.figure(figsize=(10, 6))
    plt.plot(list(results.keys()), list(results.values()), marker='o', linewidth=2, markersize=10, color='purple')
    plt.xlabel('Context Length', fontsize=12)
    plt.ylabel('MAE', fontsize=12)
    plt.title('Impact of Context Length on Chronos-2 Accuracy', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    best_ctx = min(results, key=results.get)
    print(f"\n✅ Best context length: {best_ctx} (MAE: {results[best_ctx]:.4f})")

## 💾 Save Model (Configuration Only)

In [ ]:
# Note: Chronos-2 is pre-trained, we don't save weights
# Just save the model configuration

model_info = {
    'model_type': 'chronos',
    'model_size': 'small',
    'recommended_context': best_ctx if 'best_ctx' in locals() else 96,
    'recommended_samples': 20
}

import json
config_path = '../trained_models/chronos_config.json'
os.makedirs('../trained_models', exist_ok=True)

with open(config_path, 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"Configuration saved to {config_path}")
print("\nNote: Chronos-2 uses pre-trained weights from HuggingFace.")

## 🎯 Key Takeaways

### Chronos-2 Advantages
1. **Probabilistic**: Get confidence intervals, not just point predictions
2. **Zero-Shot**: No training needed - works immediately
3. **Pre-trained**: Learned from 100K+ diverse time series
4. **Uncertainty**: Quantify prediction confidence
5. **Flexible**: Multiple model sizes for speed/accuracy tradeoff

### When to Use Chronos-2
- ✅ **Risk assessment**: Need to know prediction uncertainty
- ✅ **Decision making**: Want confidence intervals
- ✅ **Limited data**: Few training samples available
- ✅ **Quick baseline**: No time for training
- ✅ **Ensemble**: Combine with other models

### Chronos-2 vs Other Foundation Models

| Feature | Chronos-2 | TimesFM |
|---------|-----------|----------|
| Architecture | Encoder (BERT-style) | Decoder (GPT-style) |
| Uncertainty | ✅ Quantiles | ❌ Limited |
| Speed | Fast | Fast |
| Best For | Risk/uncertainty | Point forecasts |

### Model Size Selection
- **tiny/mini**: Quick experiments, prototyping
- **small**: Production baseline, good accuracy ⭐
- **base**: Higher accuracy, more compute
- **large**: Maximum accuracy, significant compute

### Tips for Best Results
1. **num_samples**: Use 20-50 for good uncertainty estimates
2. **Context length**: Longer context often helps (64-128)
3. **Quantiles**: Use [0.1, 0.5, 0.9] for 80% intervals
4. **Ensemble**: Combine with task-specific models

### Next Steps
1. Try different model sizes
2. Experiment with num_samples for uncertainty
3. Compare with TimesFM
4. Use in hybrid model with PatchTST
5. Combine in ensemble for best results